In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 20.2 MB/s eta 0:00:00


In [2]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [3]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:757: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import pandas as pd

In [10]:
speech=[]
with open('/content/drive/MyDrive/SSMT-Audios/wav.scp') as f:
    for line in f:
        speech.append(librosa.load('/content/drive/MyDrive/SSMT-Audios/Audio/'+line.split()[0]+'.wav')[0])

In [13]:
len(speech)

1741

In [15]:
transcription = pd.read_csv('/content/drive/MyDrive/SSMT-Audios/Transcript.csv',header=None)

In [32]:
transcription

,0,1,2
0,"abstract_datatypes,_classes_and_objects_[D_znh...",We have seen how to implement data structures ...,NaN
1,"abstract_datatypes,_classes_and_objects_[D_znh...",built in list type of Python. It turns out tha...,NaN
2,"abstract_datatypes,_classes_and_objects_[D_znh...","in python and create our own data types. So, w...",NaN
3,"abstract_datatypes,_classes_and_objects_[D_znh...",weeks' lectures. Let us revisit what we lean b...,NaN
4,"abstract_datatypes,_classes_and_objects_[D_znh...",A data structure is basically an organization ...,NaN
...,...,...,...
8710,"wrap_up,_python_vs_other_languages_[dXVzBXhyMx...",you can phrase your instructions. And then wor...,NaN
8711,"wrap_up,_python_vs_other_languages_[dXVzBXhyMx...",It is a mistake to sit and learn a programming...,NaN
8712,"wrap_up,_python_vs_other_languages_[dXVzBXhyMx...",you learn features of a programming language a...,NaN
8713,"wrap_up,_python_vs_other_languages_[dXVzBXhyMx...","programs. So, with this I wish you all the bes...",NaN


In [55]:
target = []
with open('/content/drive/MyDrive/SSMT-Audios/wav.scp') as f:
    for line in f:
      if((transcription[0]==line.split()[0]).any()):
        index_value = (transcription[0]==line.split()[0]).idxmax()
        target.append(transcription[1][index_value])

In [57]:
len(target)

1741

In [58]:
for i in range(len(target)):
  target[i]=target[i].upper()

In [59]:
target

['LIST IT DOES NOT MEAN THAT THE FUNCTIONS THAT ARE DEFINED FOR LISTS ARE ACTUALLY LEGAL FOR',
 'YOU WOULD HAVE FOR INSTANCE A FUNCTION LIKE SAY PUSH DEFINE AND IT WILL HAVE TWO PARAMETERS',
 'VALUE L PASSED TO IT NOW THIS CALLS THIS FUNCTION INIT WHICH IS WHY IT IS CALLED INIT. SO, INIT',
 'DELETE MAX DEFINE FOR HEAPS, BUT IT IS LIKE WE HAVE THE SEPARATE COPY OF THIS FUNCTION',
 'MAX NO MATTER HOW THE PRIORITY QUEUE IS ACTUALLY IMPLEMENTED INTERNALLY THE DELETE MAX SHOULD',
 'THEY HAVE A CONCRETE DATA TYPES WHICH WE USE IN OUR PROGRAM. ',
 'NOT WANT TO LOOK AT THE IMPLEMENTATION AND ASK WHETHER IT IS A LIST OR NOT, BECAUSE WE',
 "WEEKS' LECTURES. LET US REVISIT WHAT WE LEAN BY A DATA STRUCTURE.",
 'U. THE IMPORTANT THING IS THAT WE WOULD LIKE',
 'TO THE IMPLEMENTATION, BUT WE DEFINITELY DO NOT WANT THE IMPLEMENTATION TO DETERMINE HOW',
 'THAT THE USER SEES AS A SEQUENCE OF INSERTS AND DELETE MAX THE USER DOES NOT SEE ANY DIFFERENCE',
 'A PERCEPTION THAT THE ONE IS FASTER THAN THE OTHE

In [60]:
preds = []
for i in range(len(speech)):
    input_values = tokenizer(speech[i], return_tensors = 'pt').input_values
    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim =-1)

    preds.append(tokenizer.decode(predicted_ids[0]))

In [61]:
pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 9.4 MB/s eta 0:00:00


In [62]:
import torchmetrics

In [63]:
eval = torchmetrics.functional.word_error_rate(preds=preds, target=target)

In [64]:
target[9]

'TO THE IMPLEMENTATION, BUT WE DEFINITELY DO NOT WANT THE IMPLEMENTATION TO DETERMINE HOW'

In [65]:
preds[9]

'IPLEMATICION BUT WI DEFINITELY TO WANT WANT THE IMPLEMENT DATIO TO DETERMAN'

In [66]:
eval

tensor(0.8744)